In [ ]:
### Imp Note: Git pull utils.py from https://github.com/fastai/courses/tree/master/deeplearning1/nbs

In [1]:
%matplotlib inline
from imp import reload

from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1070 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [2]:
path = "F:/AV/Indian Actors Age/train_DETg9GD/final_folder_structure/"

In [3]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:
batch_size=16

In [5]:
# batches shuffle must be set to False when pre-computing features -- just reminder here since we will use precomputed features later on

batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 15360 images belonging to 3 classes.
Found 4546 images belonging to 3 classes.


In [6]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 15360 images belonging to 3 classes.
Found 4546 images belonging to 3 classes.
Found 6636 images belonging to 1 classes.


In [7]:
### CNN

def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dense(3, activation='softmax')
        ])

    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy']) ## for 1st try lr ws 1e-4
    model.fit_generator(batches, batches.samples, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.samples)
    #model.optimizer.lr = 0.001
    #model.fit_generator(batches, batches.samples, nb_epoch=4, validation_data=val_batches, 
    #                 nb_val_samples=val_batches.samples)
    return model

In [ ]:
model = conv1(batches)

C:\Users\ADMIN\Anaconda3\envs\Tensorflow1\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  
C:\Users\ADMIN\Anaconda3\envs\Tensorflow1\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if __name__ == '__main__':
C:\Users\ADMIN\Anaconda3\envs\Tensorflow1\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., 15360, validation_data=<keras.pre..., epochs=2, validation_steps=4546)`


Epoch 1/2
15360/15360 [==============================] - 1748s - loss: 0.3613 - acc: 0.8699 - val_loss: 0.6740 - val_acc: 0.7546
Epoch 2/2
 8347/15360 [===============>..............] - ETA: 657s - loss: 0.0769 - acc: 0.9830

In [ ]:
k = 1
test_batches = get_batches(path+'test', batch_size=k, shuffle=False)
conv_test_feat = model.predict_generator(test_batches, test_batches.samples)

In [ ]:
## middle is 0, old :1, young:2

In [ ]:
## recheck the order of classess as they appear in prediction
classes = sorted(batches.class_indices, key=batches.class_indices.get)
classes

In [ ]:
## grab IDs
ID = []
for i in range(len(test_batches.filenames)):
    temper = (test_batches.filenames)[i].split("\\",1)[1]
    ID.append(temper)
ID = pd.DataFrame({"ID":ID})
ID.head(10)

In [ ]:
## grab predictions
preds = np.argmax(conv_test_feat, axis=1)
preds = pd.DataFrame({'Class':preds})
preds['ID'] = ID
preds

In [ ]:
## append class labels to categorical class predictions
Class_upd = []
for i in range(len(preds)):
    if preds.Class[i] == 0:
        a = 'MIDDLE'
    elif preds.Class[i] == 1:
        a = "OLD"
    else:
        a = "YOUNG"
    Class_upd.append(a)
Class_upd        

In [ ]:
# file to submit
preds['Class'] = Class_upd
preds.to_csv('Simple_CNN_morelr_2nd.csv', index=False)

In [ ]:
## extra vis for checking predictions
test_batches = get_batches(path+'test', batch_size=8, shuffle=False)
imgs, labels = next(test_batches)
plots(imgs, titles = labels)